In [18]:
from collections import deque
import numpy as np
import argparse 
import os
import time
import gym
from gym import wrappers, logger



In [19]:
logger.setLevel(logger.INFO)
env = gym.make("FrozenLake-v1")

INFO: Making new env: FrozenLake-v1


In [20]:
class QParameters():
    def __init__(self):
        self.learning_rate = 0.8           # Learning rate
        self.max_steps = 99                # Max steps per episode
        self.gamma = 0.9                  # Discounting rate

        # Exploration parameters
        self.epsilon = 1.0                 # Exploration rate
        self.max_epsilon = 1.0             # Exploration probability at start
        self.min_epsilon = 0.01            # Minimum exploration probability 
        self.decay_rate = 0.00005           # Exponential decay rate for exploration prob

In [21]:
action_space = env.action_space
state_space = env.observation_space
demo = False
total_episodes = 40000        # Total episodes


In [22]:
# debug dir
outdir = "/tmp/q-learning-%s" %env
class QAgent():
    def __init__(self, observation_space, action_space, episodes,
                 demo = False):
        
        self.action_space = action_space
        self.observation_space = observation_space
        #Generate empty Q_table
        ## number of columns is equal to number of actions
        col = action_space.n
        ## number of row is equal to number of states
        row = observation_space.n
        self.q_table = np.zeros([row, col])
        
        #Set Q-Learning Parameters
        self.qParams = QParameters()
        # discount factior
        self.gamma = self.qParams.gamma
        # Exploration parameters
        self.epsilon = self.qParams.epsilon
        self.epsilon_decay = self.qParams.min_epsilon / self.epsilon
        self.epsilon_decay = self.epsilon_decay ** (self.qParams.decay_rate)

        self.learning_rate = self.qParams.learning_rate
        # file where Q Table is saved on/restored fr
        self.filename = 'q-frozenlake.npy'

        # demo or train mode 
        self.demo = demo
        # if demo mode, no exploration
        if demo:
            self.epsilon = 0
    
    def act(self, state, is_explore=False):
        if is_explore or np.random.rand() < self.epsilon:
            return self.action_space.sample()
        
        action = np.argmax(self.q_table[state, :])
        return action
    
    
    def update_q_table (self, state, action, reward, next_state):
        # Q(s, a) += 
        # alpha * (reward + gamma * max_a' Q(s', a') - Q(s, a))
        q_value = self.gamma * np.amax(self.q_table[next_state])
        q_value += reward
        q_value -= self.q_table[state, action]
        q_value *= self.learning_rate
        q_value += self.q_table[state, action]
        self.q_table[state, action] = q_value
    
    def print_q_table(self):
        print(self.q_table)
        print("Epsilon : ", self.epsilon)
        
    def save_q_table(self):
        """save trained Q Table"""
        np.save(self.filename, self.q_table)

    def load_q_table(self):
        """load trained Q Table"""
        self.q_table = np.load(self.filename)
    
    def update_epsilon(self):
        """adjust epsilon"""
        if self.epsilon > self.qParams.min_epsilon:
            self.epsilon *= self.epsilon_decay
    

In [31]:
demo = False
isExplore = False

In [34]:
# number of times the Goal state is reached
wins = 0

# instantiate a Q Learning agent
agent = QAgent(env.observation_space,
               env.action_space,
               demo=demo,
               episodes=total_episodes)

if demo:
    agent.load_q_table()


for episode in range(total_episodes):
    state = env.reset()
    done = False
    while not done:
        action = agent.act(state, is_explore=isExplore)
        next_state, reward,  done, _ = env.step(action)
        if done: 
            if reward> 0:
                wins +=1
        if not demo:
            agent.update_q_table(state,
                                 action, 
                                 reward, 
                                 next_state)
            agent.update_epsilon()
        elif demo:
            env.render()
            
        state = next_state
        
    percent_wins = 100.0 * wins / (episode + 1)
    print("-------%0.2f%% Goals in %d Episodes---------" % (percent_wins, episode))

print("Episodes: ", episode+1)
print("Goals/Holes: %d/%d" % (wins, total_episodes - wins))

agent.print_q_table()
if not demo and not isExplore:
    agent.save_q_table()
# close the env and write monitor result info to disk
env.close()  


-------0.00% Goals in 0 Episodes---------
-------0.00% Goals in 1 Episodes---------
-------0.00% Goals in 2 Episodes---------
-------0.00% Goals in 3 Episodes---------
-------0.00% Goals in 4 Episodes---------
-------0.00% Goals in 5 Episodes---------
-------0.00% Goals in 6 Episodes---------
-------0.00% Goals in 7 Episodes---------
-------0.00% Goals in 8 Episodes---------
-------0.00% Goals in 9 Episodes---------
-------0.00% Goals in 10 Episodes---------
-------0.00% Goals in 11 Episodes---------
-------0.00% Goals in 12 Episodes---------
-------0.00% Goals in 13 Episodes---------
-------0.00% Goals in 14 Episodes---------
-------0.00% Goals in 15 Episodes---------
-------0.00% Goals in 16 Episodes---------
-------0.00% Goals in 17 Episodes---------
-------0.00% Goals in 18 Episodes---------
-------0.00% Goals in 19 Episodes---------
-------0.00% Goals in 20 Episodes---------
-------0.00% Goals in 21 Episodes---------
-------0.00% Goals in 22 Episodes---------
-------0.00% Goals in

-------0.42% Goals in 235 Episodes---------
-------0.42% Goals in 236 Episodes---------
-------0.42% Goals in 237 Episodes---------
-------0.42% Goals in 238 Episodes---------
-------0.42% Goals in 239 Episodes---------
-------0.41% Goals in 240 Episodes---------
-------0.41% Goals in 241 Episodes---------
-------0.41% Goals in 242 Episodes---------
-------0.41% Goals in 243 Episodes---------
-------0.41% Goals in 244 Episodes---------
-------0.41% Goals in 245 Episodes---------
-------0.40% Goals in 246 Episodes---------
-------0.40% Goals in 247 Episodes---------
-------0.40% Goals in 248 Episodes---------
-------0.40% Goals in 249 Episodes---------
-------0.40% Goals in 250 Episodes---------
-------0.40% Goals in 251 Episodes---------
-------0.40% Goals in 252 Episodes---------
-------0.39% Goals in 253 Episodes---------
-------0.39% Goals in 254 Episodes---------
-------0.39% Goals in 255 Episodes---------
-------0.39% Goals in 256 Episodes---------
-------0.39% Goals in 257 Episod

-------2.21% Goals in 588 Episodes---------
-------2.20% Goals in 589 Episodes---------
-------2.20% Goals in 590 Episodes---------
-------2.20% Goals in 591 Episodes---------
-------2.19% Goals in 592 Episodes---------
-------2.19% Goals in 593 Episodes---------
-------2.18% Goals in 594 Episodes---------
-------2.18% Goals in 595 Episodes---------
-------2.18% Goals in 596 Episodes---------
-------2.17% Goals in 597 Episodes---------
-------2.17% Goals in 598 Episodes---------
-------2.17% Goals in 599 Episodes---------
-------2.16% Goals in 600 Episodes---------
-------2.16% Goals in 601 Episodes---------
-------2.16% Goals in 602 Episodes---------
-------2.15% Goals in 603 Episodes---------
-------2.31% Goals in 604 Episodes---------
-------2.31% Goals in 605 Episodes---------
-------2.31% Goals in 606 Episodes---------
-------2.30% Goals in 607 Episodes---------
-------2.30% Goals in 608 Episodes---------
-------2.30% Goals in 609 Episodes---------
-------2.29% Goals in 610 Episod

-------3.39% Goals in 883 Episodes---------
-------3.50% Goals in 884 Episodes---------
-------3.50% Goals in 885 Episodes---------
-------3.49% Goals in 886 Episodes---------
-------3.49% Goals in 887 Episodes---------
-------3.49% Goals in 888 Episodes---------
-------3.48% Goals in 889 Episodes---------
-------3.48% Goals in 890 Episodes---------
-------3.48% Goals in 891 Episodes---------
-------3.47% Goals in 892 Episodes---------
-------3.58% Goals in 893 Episodes---------
-------3.58% Goals in 894 Episodes---------
-------3.57% Goals in 895 Episodes---------
-------3.57% Goals in 896 Episodes---------
-------3.67% Goals in 897 Episodes---------
-------3.67% Goals in 898 Episodes---------
-------3.67% Goals in 899 Episodes---------
-------3.66% Goals in 900 Episodes---------
-------3.66% Goals in 901 Episodes---------
-------3.65% Goals in 902 Episodes---------
-------3.65% Goals in 903 Episodes---------
-------3.65% Goals in 904 Episodes---------
-------3.64% Goals in 905 Episod

-------5.61% Goals in 1140 Episodes---------
-------5.60% Goals in 1141 Episodes---------
-------5.60% Goals in 1142 Episodes---------
-------5.68% Goals in 1143 Episodes---------
-------5.68% Goals in 1144 Episodes---------
-------5.76% Goals in 1145 Episodes---------
-------5.84% Goals in 1146 Episodes---------
-------5.92% Goals in 1147 Episodes---------
-------5.92% Goals in 1148 Episodes---------
-------5.91% Goals in 1149 Episodes---------
-------5.99% Goals in 1150 Episodes---------
-------6.08% Goals in 1151 Episodes---------
-------6.07% Goals in 1152 Episodes---------
-------6.07% Goals in 1153 Episodes---------
-------6.06% Goals in 1154 Episodes---------
-------6.14% Goals in 1155 Episodes---------
-------6.14% Goals in 1156 Episodes---------
-------6.22% Goals in 1157 Episodes---------
-------6.21% Goals in 1158 Episodes---------
-------6.21% Goals in 1159 Episodes---------
-------6.20% Goals in 1160 Episodes---------
-------6.20% Goals in 1161 Episodes---------
-------6.2

...

-------43.13% Goals in 39196 Episodes---------
-------43.13% Goals in 39197 Episodes---------
-------43.13% Goals in 39198 Episodes---------
-------43.13% Goals in 39199 Episodes---------
-------43.13% Goals in 39200 Episodes---------
-------43.13% Goals in 39201 Episodes---------
-------43.13% Goals in 39202 Episodes---------
-------43.13% Goals in 39203 Episodes---------
-------43.12% Goals in 39204 Episodes---------
-------43.13% Goals in 39205 Episodes---------
-------43.12% Goals in 39206 Episodes---------
-------43.13% Goals in 39207 Episodes---------
-------43.13% Goals in 39208 Episodes---------
-------43.13% Goals in 39209 Episodes---------
-------43.13% Goals in 39210 Episodes---------
-------43.13% Goals in 39211 Episodes---------
-------43.13% Goals in 39212 Episodes---------
-------43.13% Goals in 39213 Episodes---------
-------43.13% Goals in 39214 Episodes---------
-------43.13% Goals in 39215 Episodes---------
-------43.13% Goals in 39216 Episodes---------
-------43.13%

-------43.06% Goals in 39391 Episodes---------
-------43.06% Goals in 39392 Episodes---------
-------43.06% Goals in 39393 Episodes---------
-------43.06% Goals in 39394 Episodes---------
-------43.06% Goals in 39395 Episodes---------
-------43.06% Goals in 39396 Episodes---------
-------43.06% Goals in 39397 Episodes---------
-------43.06% Goals in 39398 Episodes---------
-------43.06% Goals in 39399 Episodes---------
-------43.05% Goals in 39400 Episodes---------
-------43.06% Goals in 39401 Episodes---------
-------43.06% Goals in 39402 Episodes---------
-------43.06% Goals in 39403 Episodes---------
-------43.06% Goals in 39404 Episodes---------
-------43.05% Goals in 39405 Episodes---------
-------43.05% Goals in 39406 Episodes---------
-------43.05% Goals in 39407 Episodes---------
-------43.05% Goals in 39408 Episodes---------
-------43.05% Goals in 39409 Episodes---------
-------43.05% Goals in 39410 Episodes---------
-------43.05% Goals in 39411 Episodes---------
-------43.05%

-------43.03% Goals in 39583 Episodes---------
-------43.03% Goals in 39584 Episodes---------
-------43.03% Goals in 39585 Episodes---------
-------43.03% Goals in 39586 Episodes---------
-------43.03% Goals in 39587 Episodes---------
-------43.03% Goals in 39588 Episodes---------
-------43.03% Goals in 39589 Episodes---------
-------43.03% Goals in 39590 Episodes---------
-------43.03% Goals in 39591 Episodes---------
-------43.03% Goals in 39592 Episodes---------
-------43.03% Goals in 39593 Episodes---------
-------43.03% Goals in 39594 Episodes---------
-------43.03% Goals in 39595 Episodes---------
-------43.03% Goals in 39596 Episodes---------
-------43.03% Goals in 39597 Episodes---------
-------43.03% Goals in 39598 Episodes---------
-------43.03% Goals in 39599 Episodes---------
-------43.03% Goals in 39600 Episodes---------
-------43.03% Goals in 39601 Episodes---------
-------43.03% Goals in 39602 Episodes---------
-------43.03% Goals in 39603 Episodes---------
-------43.03%

-------43.02% Goals in 39774 Episodes---------
-------43.02% Goals in 39775 Episodes---------
-------43.02% Goals in 39776 Episodes---------
-------43.02% Goals in 39777 Episodes---------
-------43.02% Goals in 39778 Episodes---------
-------43.02% Goals in 39779 Episodes---------
-------43.02% Goals in 39780 Episodes---------
-------43.02% Goals in 39781 Episodes---------
-------43.02% Goals in 39782 Episodes---------
-------43.02% Goals in 39783 Episodes---------
-------43.02% Goals in 39784 Episodes---------
-------43.02% Goals in 39785 Episodes---------
-------43.02% Goals in 39786 Episodes---------
-------43.02% Goals in 39787 Episodes---------
-------43.02% Goals in 39788 Episodes---------
-------43.02% Goals in 39789 Episodes---------
-------43.02% Goals in 39790 Episodes---------
-------43.02% Goals in 39791 Episodes---------
-------43.02% Goals in 39792 Episodes---------
-------43.02% Goals in 39793 Episodes---------
-------43.02% Goals in 39794 Episodes---------
-------43.02%

-------43.06% Goals in 39955 Episodes---------
-------43.06% Goals in 39956 Episodes---------
-------43.06% Goals in 39957 Episodes---------
-------43.06% Goals in 39958 Episodes---------
-------43.06% Goals in 39959 Episodes---------
-------43.06% Goals in 39960 Episodes---------
-------43.06% Goals in 39961 Episodes---------
-------43.06% Goals in 39962 Episodes---------
-------43.06% Goals in 39963 Episodes---------
-------43.06% Goals in 39964 Episodes---------
-------43.06% Goals in 39965 Episodes---------
-------43.06% Goals in 39966 Episodes---------
-------43.06% Goals in 39967 Episodes---------
-------43.06% Goals in 39968 Episodes---------
-------43.06% Goals in 39969 Episodes---------
-------43.06% Goals in 39970 Episodes---------
-------43.06% Goals in 39971 Episodes---------
-------43.06% Goals in 39972 Episodes---------
-------43.06% Goals in 39973 Episodes---------
-------43.06% Goals in 39974 Episodes---------
-------43.06% Goals in 39975 Episodes---------
-------43.06%

In [36]:
demo = True
isExplore = False

In [39]:
# number of times the Goal state is reached
wins = 0

if demo:
        agent.load_q_table()


for episode in range(50):
    state = env.reset()
    done = False
    while not done:
        action = agent.act(state, is_explore=isExplore)
        next_state, reward,  done, _ = env.step(action)
        if done: 
            if reward> 0:
                wins +=1
        if not demo:
            agent.update_q_table(state,
                                 action, 
                                 reward, 
                                 next_state)
            agent.update_epsilon()
        elif demo:
            env.render()
            
        state = next_state
        
    percent_wins = 100.0 * wins / (episode + 1)
    print("-------%0.2f%% Goals in %d Episodes---------" % (percent_wins, episode))

print("Episodes: ", episode+1)
print("Goals/Holes: %d/%d" % (wins, 50 - wins))

agent.print_q_table()
if not demo and not isExplore:
    agent.save_q_table()
# close the env and write monitor result info to disk
env.close()  


  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
-------0.00% Goals in 0 Episodes---------
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
-------50.00% Goals in 1 Episodes---------
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)

  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
-------21.88% Goals in 31 Episodes---------
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
-------24.24% Goals in 32 Episodes---------
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
F

In [35]:
agent.load_q_table()
agent.print_q_table()

[[1.32127485e-02 5.05877333e-03 1.37792260e-01 8.77246643e-03]
 [3.80562466e-02 1.31538712e-03 3.87983090e-03 1.33222356e-02]
 [2.18146553e-03 2.54630904e-02 9.25243388e-03 7.97874938e-03]
 [9.07351703e-03 8.20833759e-04 2.95689430e-05 4.50439974e-02]
 [9.67387001e-02 4.41311313e-03 7.06345191e-05 3.98394782e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.91688410e-04 1.28911371e-06 9.53341581e-02 4.39760128e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.59360019e-05 4.51330352e-04 9.89171823e-03 4.27620005e-02]
 [8.31522362e-03 4.56772217e-01 8.81004993e-04 4.15190474e-03]
 [6.97676172e-01 1.39686669e-03 1.63128441e-03 1.14767466e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.33810475e-02 7.80834741e-05 2.75595324e-01 5.71166812e-02]
 [2.03422490e-01 8.65629805e-01 3.63453281e-02 8.66063079e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.000000